<a href="https://colab.research.google.com/github/jchanlau/NLP_colab/blob/main/nlp_pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pilot program to distill text from PDF and perform sentiment analysis**

In [1]:
!pip install transformers        # Hugging Face transfomers
!pip install --upgrade pymupdf   # util to manipulate PDF files
!pip install nltk
# !pip install PyPDF2            # alternative 

from transformers import pipeline # import Hugging Face
import fitz # pymupdf
import nltk # parse sentences in paragraph
nltk.download('punkt')
import numpy as np; import pandas as pd  # standard libraries
import os   # in Google Colab, the home directory is content, 
            # need to store data files in content/data


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Useful functions

# find text between first and last value, avoids repetitions
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

# find text between first and last value
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def parse_sentences(text, classifier, min_chars = 50):
  sentences = nltk.sent_tokenize(text)
  sentence_list = []
  sentiment_list = []
  for s in sentences:
    if len(s)> min_chars:
      sentiment_list.append(pd.DataFrame(classifier(s)))
      sentence_list.append(s) 
  return pd.concat(sentiment_list).reset_index(), sentence_list

def examine_sentence(sentiment_list, sentence_list, sentiment):
  idx = sentiment_list.index[sentiment_list.label==sentiment].tolist()
  return [sentence_list[i] for i in idx]



In [3]:
# get text
doc = fitz.open("data/1PEREA2021001.pdf")
text= ""
for page in doc:
    text += page.get_text()
doc.close()

context = find_between_r(text, "CONTEXT", "RECENT DEVELOPMENTS")
#print(context)
#context_sentences = nltk.sent_tokenize(context_section)
#context_sentences = parse_sentences(context)


In [4]:
classifier = pipeline("sentiment-analysis")

sentiment_list, sentence_list = parse_sentences(context, classifier)

#for s in context_sentences:
#  if len(s)>10:
#    print(s)
#    print(pd.DataFrame(classifier(s)))



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
examine_sentence(sentiment_list, sentence_list, 'NEGATIVE')

['Prudent fiscal management, supported by fiscal rules, has helped contain government \ndebt within the 30-percent-of-GDP ceiling until 2019, at the lowest level in the region.',
 'On the heels of a combination of adverse shocks that slowed \nthe growth momentum in recent years, the pandemic presented an unprecedented challenge, taking \na heavy toll on lives and livelihoods and causing one of the deepest recessions.',
 'The authorities expect to treat the arrangement as precautionary.',
 'A new Congress was elected in January 2020, but tensions with the executive power \ncontinued to affect institutional cohesion.',
 'Instability culminated in November 2020 when the former \npresident was impeached based on corruption allegations.',
 'His immediate successor resigned \nfollowing deadly protests within days after his confirmation, following which Francisco Sagasti was \ninstalled as the interim president.']